In [1]:
import pandas as pd
import sys
sys.path.append("..")
import utility.utility_functions as ut

In [2]:
week_end = ['Saturday','Sunday']
mid_week = ['Monday','Tuesday','Wednesday','Thursday','Friday']

In [3]:
midnight = [
            '00:00 - 00:59',
            '01:00 - 01:59',
            '02:00 - 02:59',
            '03:00 - 03:59',
            '04:00 - 04:59',
            '05:00 - 05:59',
            '06:00 - 06:59'
            ]
morning = [
            '07:00 - 07:59',
            '08:00 - 08:59',
            '09:00 - 09:59',
            '10:00 - 10:59',
            '11:00 - 11:59'
            ]
noon_afternoon = [
            '12:00 - 12:59',
            '13:00 - 13:59',
            '14:00 - 14:59',
            '15:00 - 15:59',
            '16:00 - 16:59',
            '17:00 - 17:59',
            '18:00 - 18:59',
            '19:00 - 19:59'
            ]
night = [
            '20:00 - 20:59',
            '21:00 - 21:59',
            '22:00 - 22:59',
            '23:00 - 23:59'
]

In [4]:
df = ut.load_post_data_prep_all_zone()
df.dropna(inplace=True)
df = df.drop(df[df.trip_time < 5].index)
df = df.drop('pickup_date',axis=1)
df.reset_index(inplace=True,drop=True)

In [5]:
df.head()

,weekday,time_binned,Tmax,Tmin,Tavg,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis,Zone
0,Tuesday,00:00 - 00:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41,4
1,Tuesday,01:00 - 01:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78,4
2,Tuesday,02:00 - 02:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62,4
3,Tuesday,03:00 - 03:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65,4
4,Tuesday,04:00 - 04:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30,4


In [6]:
df.drop(['trip_distance','trip_time','speed'],axis=1,inplace=True)

In [17]:
import seaborn as sns
import plotly.express as px
import numpy as np

corr = df.corr().abs()

fig = px.imshow(corr, text_auto=True)
fig.update_layout(
    autosize=False,
    width=800,
    height=500,)

fig.show()

In [18]:
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]
to_drop
df1 = df.drop(to_drop, axis=1)


In [19]:
to_drop

['Tmin', 'Tavg', 'HDD']

In [10]:
main_df = ut.swap_first_and_last_col(df1)

In [11]:
mid_week_morning_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(morning))]
mid_week_night_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(night))]
mid_week_noon_afternoon_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(noon_afternoon))]
mid_week_midnight_df = main_df.loc[(main_df['weekday'].isin(mid_week)) & (main_df['time_binned'].isin(midnight))]


In [12]:
week_end_morning_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(morning))]
week_end_night_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(night))]
week_end_noon_afternoon_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(noon_afternoon))]
week_end_midnight_df = main_df.loc[(main_df['weekday'].isin(week_end)) & (main_df['time_binned'].isin(midnight))]

In [13]:
df_dict =  {
                'mid_week_morning_df':mid_week_morning_df,
                'mid_week_night_df':mid_week_night_df,
                'mid_week_noon_afternoon_df':mid_week_noon_afternoon_df,
                'mid_week_midnight_df':mid_week_midnight_df,
                'week_end_morning_df':week_end_morning_df,
                'week_end_night_df':week_end_night_df,
                'week_end_noon_afternoon_df':week_end_noon_afternoon_df,
                'week_end_midnight_df':week_end_midnight_df
}

In [14]:
import rmse_script 

C:\Users\barka\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\compat.py:36: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



In [15]:
for key, value in df_dict.items():
    x,y,t = rmse_script.get_rmse_for_df(value,key)
    print(f'Group {key} RMSE of {x:.2f} and test RMSE of {t:.2f} target feature std of {y:.2f}')

Group mid_week_morning_df RMSE of 110.03 and test RMSE of 109.87 target feature std of 132.55
Group mid_week_night_df RMSE of 144.98 and test RMSE of 144.39 target feature std of 153.70
Group mid_week_noon_afternoon_df RMSE of 131.50 and test RMSE of 130.73 target feature std of 159.56
Group mid_week_midnight_df RMSE of 45.92 and test RMSE of 46.22 target feature std of 49.47
Group week_end_morning_df RMSE of 79.86 and test RMSE of 79.17 target feature std of 89.38
Group week_end_night_df RMSE of 115.19 and test RMSE of 115.52 target feature std of 120.24
Group week_end_noon_afternoon_df RMSE of 110.47 and test RMSE of 109.72 target feature std of 126.58
Group week_end_midnight_df RMSE of 100.26 and test RMSE of 101.91 target feature std of 115.24
